In [3]:
pip install opencv-python mediapipe numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Phase 1: Notebook Setup & Initialization (Local Jupyter Version)

# Ensure libraries are installed: pip install opencv-python mediapipe numpy
import cv2
import mediapipe as mp
import numpy as np
import math
import os
import time

print("Libraries Imported Successfully!")

# --- MediaPipe Pose Configuration ---
mp_pose = mp.solutions.pose
POSE_MODEL_COMPLEXITY = 1       # 0=lite, 1=full, 2=heavy
POSE_SMOOTH_LANDMARKS = True    # Reduce jitter
POSE_ENABLE_SEGMENTATION = False
POSE_MIN_DETECTION_CONFIDENCE = 0.5
POSE_MIN_TRACKING_CONFIDENCE = 0.5

# --- Smoothing Configuration ---
SMOOTHING_ALPHA = 0.1 # Exponential Moving Average factor (smaller=smoother, more lag)

# --- Visualization Configuration ---
COLOR_RED = (0, 0, 255)
COLOR_GREEN = (0, 255, 0)
COLOR_BLUE = (255, 0, 0)
COLOR_YELLOW = (0, 255, 255)
COLOR_WHITE = (255, 255, 255)
COLOR_BLACK = (0, 0, 0)
COLOR_LIGHT_BLUE = (255, 100, 0)
COLOR_PURPLE = (255, 0, 255)
COLOR_ORANGE = (0, 165, 255)
FONT = cv2.FONT_HERSHEY_SIMPLEX

# --- Live Feed Settings ---
CAMERA_INDEX = 0 # Default camera index
ESTIMATED_FPS = 30.0 # Use an estimate for initial dt, will be updated

print("\nPhase 1 Setup Complete.")

Libraries Imported Successfully!

Phase 1 Setup Complete.


In [5]:
# Phase 3: Biomechanical Logic Functions

# --- Helper Functions ---
def detect_stance(landmarks):
    if landmarks is None: return "unknown"
    # Assuming bowler is roughly in -Z direction relative to hip origin
    left_shoulder_z = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z
    right_shoulder_z = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z
    # Shoulder closer to origin (less negative Z) is typically back shoulder
    if left_shoulder_z > right_shoulder_z: return "right_handed"
    else: return "left_handed"

def calculate_angle_3d(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    norm_ba, norm_bc = np.linalg.norm(ba), np.linalg.norm(bc)
    if norm_ba == 0 or norm_bc == 0: return 0.0
    cosine_angle = np.clip(np.dot(ba, bc) / (norm_ba * norm_bc), -1.0, 1.0)
    return np.degrees(np.arccos(cosine_angle))

# --- Metric Functions ---
def get_weight_transfer(landmarks, stance):
    if landmarks is None or stance == "unknown": return 50.0
    front_ankle_idx = mp_pose.PoseLandmark.LEFT_ANKLE.value if stance == "right_handed" else mp_pose.PoseLandmark.RIGHT_ANKLE.value
    back_ankle_idx = mp_pose.PoseLandmark.RIGHT_ANKLE.value if stance == "right_handed" else mp_pose.PoseLandmark.LEFT_ANKLE.value
    front_ankle_z = landmarks[front_ankle_idx].z
    back_ankle_z = landmarks[back_ankle_idx].z
    hip_center_z = 0.0
    stance_depth = abs(front_ankle_z - back_ankle_z)
    if stance_depth < 0.05: return 50.0 # Avoid division by zero/instability
    hip_position_relative_to_back = hip_center_z - back_ankle_z
    # Normalize position by stance depth along Z
    transfer_ratio = hip_position_relative_to_back / (front_ankle_z - back_ankle_z)
    return np.clip(transfer_ratio * 100, 0, 100)

def get_front_elbow_angle(landmarks, stance):
    if landmarks is None or stance == "unknown": return None
    if stance == "right_handed":
        s_idx, e_idx, w_idx = mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_ELBOW.value, mp_pose.PoseLandmark.LEFT_WRIST.value
    else: # left_handed
        s_idx, e_idx, w_idx = mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_ELBOW.value, mp_pose.PoseLandmark.RIGHT_WRIST.value
    shoulder = [landmarks[s_idx].x, landmarks[s_idx].y, landmarks[s_idx].z]
    elbow = [landmarks[e_idx].x, landmarks[e_idx].y, landmarks[e_idx].z]
    wrist = [landmarks[w_idx].x, landmarks[w_idx].y, landmarks[w_idx].z]
    return calculate_angle_3d(shoulder, elbow, wrist)

def get_bat_speed_approx(current_landmarks_np, prev_landmarks_np, stance, dt):
    if prev_landmarks_np is None or dt <= 0 or current_landmarks_np is None or stance == "unknown": return 0.0
    wrist_idx = mp_pose.PoseLandmark.LEFT_WRIST.value if stance == "right_handed" else mp_pose.PoseLandmark.RIGHT_WRIST.value
    current_wrist_pos = current_landmarks_np[wrist_idx]
    prev_wrist_pos = prev_landmarks_np[wrist_idx]
    distance = np.linalg.norm(current_wrist_pos - prev_wrist_pos) # Meters
    return distance / dt # m/s

print("Phase 3 Functions Defined.")

Phase 3 Functions Defined.


In [6]:
# Combined Phases 2, 4, 5: Pose Engine, Visualization & Smoothing for Live Feed (Local Jupyter)

# --- Initialization ---
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=POSE_MODEL_COMPLEXITY,
    smooth_landmarks=POSE_SMOOTH_LANDMARKS,
    enable_segmentation=POSE_ENABLE_SEGMENTATION,
    min_detection_confidence=POSE_MIN_DETECTION_CONFIDENCE,
    min_tracking_confidence=POSE_MIN_TRACKING_CONFIDENCE
)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe Pose Initialized.")

# Initialize Smoothed Variables
smoothed_transfer = 50.0
smoothed_angle = 90.0
smoothed_speed = 0.0
print("Smoothed metric variables initialized.")

# --- Video Input Setup (Using OpenCV for Local Camera) ---
cap = cv2.VideoCapture(CAMERA_INDEX)
if not cap.isOpened():
    print(f"Error: Could not open camera with index {CAMERA_INDEX}.")
else:
    # Try to get camera properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # Note: cap.get(cv2.CAP_PROP_FPS) often returns 0 for webcams, use dynamic dt
    print(f"Camera {CAMERA_INDEX} opened successfully (Default Resolution: {frame_width}x{frame_height}).")

    prev_world_landmarks_np = None
    frame_count = 0
    last_time = time.time() # For dynamic dt calculation

    print("Starting live video processing loop... Press ESC in the display window to Exit.")

    # --- Main Frame Processing Loop ---
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue # Skip processing if frame is empty

        frame_count += 1
        image = cv2.flip(image, 1) # Flip horizontally for a selfie-view display.

        # Calculate dynamic delta_time
        current_time = time.time()
        dt = current_time - last_time
        last_time = current_time
        live_fps = 1.0 / dt if dt > 0 else 0

        # Image Prep
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False

        # Run Pose Detection
        results = pose.process(image_rgb)

        image_rgb.flags.writeable = True
        # Keep image in BGR format for OpenCV drawing/display
        # image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR) # No need to convert back if drawing on original 'image'

        # Extract Landmarks
        current_world_landmarks = None
        current_world_landmarks_np = None

        if results.pose_world_landmarks:
            current_world_landmarks = results.pose_world_landmarks.landmark
            current_world_landmarks_np = np.array(
                [[lm.x, lm.y, lm.z] for lm in current_world_landmarks]
            )

            # --- Calculate Raw Metrics ---
            stance = detect_stance(current_world_landmarks)
            raw_transfer = get_weight_transfer(current_world_landmarks, stance)
            raw_elbow_angle = get_front_elbow_angle(current_world_landmarks, stance)
            raw_bat_speed = get_bat_speed_approx(current_world_landmarks_np, prev_world_landmarks_np, stance, dt)

            # --- Apply Smoothing ---
            smoothed_transfer = (SMOOTHING_ALPHA * raw_transfer) + ((1 - SMOOTHING_ALPHA) * smoothed_transfer)
            if raw_elbow_angle is not None:
                smoothed_angle = (SMOOTHING_ALPHA * raw_elbow_angle) + ((1 - SMOOTHING_ALPHA) * smoothed_angle)
            smoothed_speed = (SMOOTHING_ALPHA * raw_bat_speed) + ((1 - SMOOTHING_ALPHA) * smoothed_speed)

            # --- Prepare Display Values ---
            display_transfer = smoothed_transfer
            display_elbow_angle = smoothed_angle
            display_bat_speed = smoothed_speed

            # --- Visualization ---
            # Draw Skeleton (on the BGR 'image')
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image=image, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=COLOR_GREEN, thickness=1, circle_radius=2),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=COLOR_BLUE, thickness=1, circle_radius=1)
                )

            # Draw Metrics
            bar_x, bar_y, bar_w, bar_h = 20, 30, 250, 25
            fill_w = int(bar_w * (display_transfer / 100))
            back_percent = 100 - display_transfer

            cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_w, bar_y + bar_h), COLOR_BLACK, -1)
            cv2.rectangle(image, (bar_x, bar_y), (bar_x + fill_w, bar_y + bar_h), COLOR_LIGHT_BLUE, -1)
            cv2.putText(image, f"Wt:", (bar_x - 30, bar_y + 18), FONT, 0.5, COLOR_WHITE, 1, cv2.LINE_AA)
            cv2.putText(image, f"{back_percent:.0f}%", (bar_x + 5, bar_y + 18), FONT, 0.5, COLOR_WHITE, 1, cv2.LINE_AA)
            cv2.putText(image, f"{display_transfer:.0f}%", (bar_x + bar_w - 45, bar_y + 18), FONT, 0.5, COLOR_WHITE, 1, cv2.LINE_AA)
            cv2.putText(image, "BACK", (bar_x + 40, bar_y + 18), FONT, 0.5, (180, 180, 180), 1, cv2.LINE_AA)
            cv2.putText(image, "FRONT", (bar_x + bar_w - 95, bar_y + 18), FONT, 0.5, (180, 180, 180), 1, cv2.LINE_AA)

            angle_text = f"Elbow: {display_elbow_angle:.1f} deg" if raw_elbow_angle is not None else "Elbow: N/A"
            cv2.putText(image, angle_text, (bar_x, bar_y + bar_h + 30), FONT, 0.7, COLOR_GREEN, 2, cv2.LINE_AA)

            speed_text = f"Speed: {display_bat_speed:.1f} m/s"
            cv2.putText(image, speed_text, (bar_x, bar_y + bar_h + 60), FONT, 0.7, COLOR_YELLOW, 2, cv2.LINE_AA)

            stance_text = f"Stance: {stance.replace('_', ' ').title()}"
            cv2.putText(image, stance_text, (bar_x, bar_y + bar_h + 90), FONT, 0.7, COLOR_ORANGE, 2, cv2.LINE_AA)

            fps_text = f"FPS: {live_fps:.1f}"
            cv2.putText(image, fps_text, (frame_width - 100, 30), FONT, 0.6, COLOR_RED, 2, cv2.LINE_AA)

            # --- Update Previous Landmarks ---
            prev_world_landmarks_np = current_world_landmarks_np

        else: # No pose detected
            cv2.putText(image, "No Pose Detected", (10, frame_height - 10), FONT, 0.5, COLOR_RED, 1)
            # Optionally reset smoothed values if pose is lost for a while
            # smoothed_transfer, smoothed_angle, smoothed_speed = 50.0, 90.0, 0.0


        # --- Display the frame using cv2.imshow ---
        cv2.imshow('Cricket Biomechanics Analyzer - Press ESC to Exit', image)

        # --- Check for exit key (ESC) ---
        if cv2.waitKey(1) & 0xFF == 27: # waitKey(1) checks for key press for 1ms
            print("ESC key pressed. Exiting loop.")
            break


    # --- End of Loop ---
    print(f"Processed approximately {frame_count} frames.")

    # --- Release Resources ---
    cap.release()
    cv2.destroyAllWindows()
    # Attempt to close any lingering windows
    for i in range(5): cv2.waitKey(1)
    print("Camera capture and windows released.")

print("\nLive Feed Analysis Attempt Complete.")

MediaPipe Pose Initialized.
Smoothed metric variables initialized.
Camera 0 opened successfully (Default Resolution: 640x480).
Starting live video processing loop... Press ESC in the display window to Exit.
ESC key pressed. Exiting loop.
Processed approximately 308 frames.
Camera capture and windows released.

Live Feed Analysis Attempt Complete.
